# RetainX – Customer Revenue & Subscription Retention Intelligence System  
### Data Cleaning & Preprocessing  

**Client:** AirWave Communications  
**Domain:** Telecom / Subscription Analytics  
**Author:** Ujjwal Verma  

---

## Objective
This notebook performs **data cleaning and preprocessing** on the raw telecom customer dataset.

Although the source dataset is already complete (no missing values or duplicates), this phase focuses on:
- Data type standardization
- Outlier treatment
- Feature preparation
- Ensuring analytics-ready consistency

The output of this notebook serves as the **final cleaned dataset** that is ingested into PostgreSQL for SQL-based analytics and modeling.


## 1. Library Imports

This section imports the Python libraries required for preprocessing and numerical transformations.

- **Pandas** is used for data manipulation and export
- **NumPy** is used for numerical operations and outlier handling


In [ ]:
import pandas as pd
import numpy as np

## 2. Load Raw Telecom Customer Dataset

The raw telecom customer dataset is loaded from the source CSV file.

At this stage:
- The dataset is treated as the source of truth
- No assumptions are made about downstream analytics
- The focus is on preparing the data for analytical modeling


In [ ]:
df = pd.read_csv("../02_Data/raw/telecom_churn.csv")

## 3. Data Type Standardization

Certain columns are standardized to ensure compatibility with analytical workflows and SQL ingestion:

- Registration date is converted to datetime format
- Pincode is stored as a string to preserve leading zeros

This step prevents downstream inconsistencies and errors.


In [ ]:
df['date_of_registration'] = pd.to_datetime(df['date_of_registration'], errors='coerce')
df['pincode'] = df['pincode'].astype(str)

## 4. Duplicate Record Validation

Although the dataset does not contain duplicate records, a validation check is performed to confirm data integrity.

This is a standard best practice in enterprise analytics pipelines.


In [5]:
initial_count = df.shape[0]
df.drop_duplicates(subset=['customer_id'], inplace=True)
final_count = df.shape[0]

print("Duplicate rows removed:", initial_count - final_count)

Duplicate rows removed: 0


## 5. Missing Value Validation

The dataset is validated for missing values.

Since no missing values are present in the source data:
- No imputation is required
- This step serves as a formal quality check


In [6]:
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

df.isnull().sum()

/var/folders/xl/zg7xr3f931ggd0w1xs4831280000gn/T/ipykernel_70885/3432125287.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
/var/folders/xl/zg7xr3f931ggd0w1xs4831280000gn/T/ipykernel_70885/3432125287.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always

customer_id             0
telecom_partner         0
gender                  0
age                     0
state                   0
city                    0
pincode                 0
date_of_registration    0
num_dependents          0
estimated_salary        0
calls_made              0
sms_sent                0
data_used               0
churn                   0
dtype: int64

## 6. Outlier Treatment Using IQR Method

Even in clean datasets, extreme values can distort:
- Averages
- Correlations
- Segmentation thresholds

The Interquartile Range (IQR) method is applied to cap extreme values while preserving all records.


In [ ]:
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns

for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = np.where(df[col] < lower, lower, df[col])
    df[col] = np.where(df[col] > upper, upper, df[col])

## 7. Feature Preparation – Customer Tenure

A customer tenure feature (`tenure_months`) is derived using the registration date.

This feature is essential for:
- Lifecycle segmentation
- Churn risk analysis
- Retention strategy development


In [ ]:
df['tenure_months'] = ((pd.Timestamp.now() - df['date_of_registration']).dt.days / 30).astype(int)

## 8. Final Data Quality Validation

A final validation check ensures that:
- No null values exist
- No duplicates were introduced during preprocessing
- All derived features are present

This confirms readiness for SQL ingestion.


In [9]:
{
 "Dataset Shape": df.shape,
 "Null Check": df.isnull().sum().sum(),
 "Duplicate Count": df.duplicated().sum(),
 "Tenure Created": "tenure_months" in df.columns
}

{'Dataset Shape': (243553, 15),
 'Null Check': 0,
 'Duplicate Count': 0,
 'Tenure Created': True}

## 9. Export Cleaned Dataset for SQL Ingestion

The cleaned dataset is exported as a CSV file.

This file is used as:
- Input for PostgreSQL ingestion
- Foundation for SQL-based feature engineering
- Single source of truth for analytics and BI


In [ ]:
df.to_csv("../02_Data/cleaned_python/telecom_cleaned.csv", index=False)
print("Clean dataset exported successfully.")

Clean dataset exported successfully.


## Key Takeaways

- The source dataset is clean and well-structured, requiring minimal remediation.
- Preprocessing focused on standardization, outlier handling, and feature preparation rather than basic cleaning.
- The resulting dataset is analytics-ready and suitable for relational database ingestion.
- This notebook acts as the transition point between raw data and SQL-based analytical modeling.

---

## Next Step

The cleaned dataset generated here is ingested into PostgreSQL, where:
- Business feature engineering is performed
- Customer segmentation is created
- Analytical (GOLD) tables are built

➡ Proceed to **03_SQL**
